In [1]:
from pyomo.environ import *
import sys
from itertools import product

sys.path.append('../../../src')
# from energiapy.utils.scale_utils import scale_list, scale_tuple

In [2]:
T_up = 10
T = [i for i in range(0,T_up)]
# structure = [4, 7]
# time_index_dict = dict()


In [3]:
m = ConcreteModel()

In [4]:
m.time = Set(initialize=T)
m.sources = Set(initialize=['A'])
m.sinks = Set(initialize=['B'])
m.transports = Set(initialize=['truckAB', 'planeAB'])
m.resources = Set(initialize=['resource1'])

In [5]:
m.Exp = Var(m.sources, m.sinks, m.transports, m.resources, m.time, domain=NonNegativeReals)
m.Transit = Var(m.sources, m.sinks, m.transports, m.resources, m.time, domain=NonNegativeReals)

In [6]:
travel_time_dict = {('A', 'B'): {'truckAB': 5, 'planeAB': 2}}

In [7]:
# m.Exp.pprint()

In [8]:
scales = [tuple(T)]
scale_iter = list(product(*scales))

In [9]:
def constraint_transit_balance(instance: ConcreteModel) -> Constraint:
    
    def transit_balance_rule(instance: ConcreteModel, source, sink, transport, resource, *scale_list):
            
        current_index = scale_iter.index(scale_list[:0 + 1])
        difference = scale_iter.index(scale_list[:0 + 1]) - travel_time_dict[(source, sink)][transport]
        
        if current_index - 1 >= 0:
            initial = instance.Transit[source, sink, transport, resource, scale_iter[current_index - 1]]
        else:
            initial = 0
            
        incoming = instance.Exp[source, sink, transport, resource, scale_iter[current_index]]
        
        if difference >= 0:
            outgoing = instance.Exp[source, sink, transport, resource, scale_iter[difference]]
        else:
            outgoing = 0
        
        return instance.Transit[source, sink, transport, resource, scale_iter[current_index]] == initial + incoming - outgoing
    
    
    instance.constraint_transit_balance = Constraint(instance.sources, instance.sinks, instance.transports, instance.resources, *scales, rule=transit_balance_rule)
        

In [10]:
constraint_transit_balance(m)

In [11]:
m.constraint_transit_balance.pprint()

constraint_transit_balance : Size=20, Index=sources*sinks*transports*resources*{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}, Active=True
    Key                                   : Lower : Body                                                                                                                                : Upper : Active
    ('A', 'B', 'planeAB', 'resource1', 0) :   0.0 :                                                                     Transit[A,B,planeAB,resource1,0] - Exp[A,B,planeAB,resource1,0] :   0.0 :   True
    ('A', 'B', 'planeAB', 'resource1', 1) :   0.0 :                                Transit[A,B,planeAB,resource1,1] - (Transit[A,B,planeAB,resource1,0] + Exp[A,B,planeAB,resource1,1]) :   0.0 :   True
    ('A', 'B', 'planeAB', 'resource1', 2) :   0.0 : Transit[A,B,planeAB,resource1,2] - (Transit[A,B,planeAB,resource1,1] + Exp[A,B,planeAB,resource1,2] - Exp[A,B,planeAB,resource1,0]) :   0.0 :   True
    ('A', 'B', 'planeAB', 'resource1', 3) :   0.0 : Transit[A,B,planeAB,r

In [12]:
m.Transit.pprint()

Transit : Size=20, Index=sources*sinks*transports*resources*time
    Key                                   : Lower : Value : Upper : Fixed : Stale : Domain
    ('A', 'B', 'planeAB', 'resource1', 0) :     0 :  None :  None : False :  True : NonNegativeReals
    ('A', 'B', 'planeAB', 'resource1', 1) :     0 :  None :  None : False :  True : NonNegativeReals
    ('A', 'B', 'planeAB', 'resource1', 2) :     0 :  None :  None : False :  True : NonNegativeReals
    ('A', 'B', 'planeAB', 'resource1', 3) :     0 :  None :  None : False :  True : NonNegativeReals
    ('A', 'B', 'planeAB', 'resource1', 4) :     0 :  None :  None : False :  True : NonNegativeReals
    ('A', 'B', 'planeAB', 'resource1', 5) :     0 :  None :  None : False :  True : NonNegativeReals
    ('A', 'B', 'planeAB', 'resource1', 6) :     0 :  None :  None : False :  True : NonNegativeReals
    ('A', 'B', 'planeAB', 'resource1', 7) :     0 :  None :  None : False :  True : NonNegativeReals
    ('A', 'B', 'planeAB', 'resource1